In [1]:
import sys
sys.path.append('../src')
sys.path.append('../src/data/')
sys.path.append('../src/models/')
sys.path.append('../src/features/')
sys.path.append('../src/visualization/')
# will reload any library
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [11]:
from data.metrics import recall
from visualization.plot_utils import write_latex_table, summary_pooling_table, \
pooling_plots, pandas_settings, plot_settings,  save_result, glance_dict
# general
import pandas as pd
import numpy as np
from tqdm import tqdm
from features.tracks_info import TrackInfo
from collections import OrderedDict
from itertools import product
from data_utils import product_size
import matplotlib.pyplot as plt
import seaborn as sns
import compressed_pickle as cpick
def pool_stats(pid,k,strat,n_pick,true_t,pool_t,true_a,pool_a):
    r_t = recall(true_t, pool_t)
    r_a = recall(true_a, pool_a)
    return OrderedDict([('pid',pid),('k',k),('strat',strat),('n',n_pick),('recall_tracks',r_t),('recall_artist',r_a)])

plot_settings()
pandas_settings()

# create pooling model

In [44]:
turi2auri = cpick.load('../data/interim/track_uri2artist_uri.pkl.bz2')
genres = cpick.load('../data/interim/genres.pkl.bz2')
turi2genres ={}
for turi,auri in turi2auri.items():
    turi2genres[turi]=genres[auri]
cpick.save('../data/interim/track_uri2genres.pkl.bz2',turi2genres)
glance_dict(turi2genres)

Saving to ../data/interim/track_uri2genres.pkl.bz2


{'0UaMYEvWZi0ZqiDOoHU3YI': ['dance pop',
  'hip hop',
  'hip pop',
  'pop',
  'pop rap',
  'r&b',
  'southern hip hop',
  'urban contemporary'],
 '6I9VzXrHxO9rA9A5euc8Ak': ['dance pop',
  'pop',
  'post-teen pop',
  'urban contemporary'],
 '0WqIKmW4BTrj3eJFmnCKMv': ['dance pop', 'pop', 'post-teen pop', 'r&b']}

## prune and vectorize words

In [45]:
from gensim import corpora
texts = [val for val in turi2genres.values()]
## remove words that appear only once
#from collections import defaultdict
#frequency = defaultdict(int)
#for text in texts:
#    for token in text:
#        frequency[token] += 1

#texts = [[token for token in text if frequency[token] > 1] for text in texts]

## dictionary and mm data

In [46]:
from gensim import corpora
dictionary = corpora.Dictionary(texts)
dictionary.save('genres.dict')
print(glance_dict(dictionary.token2id,10))
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize('genres.mm', corpus)

2018-05-04 05:36:03,893 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-05-04 05:36:04,017 : INFO : adding document #10000 to Dictionary(766 unique tokens: ['dance pop', 'hip hop', 'hip pop', 'pop', 'pop rap']...)
2018-05-04 05:36:04,109 : INFO : adding document #20000 to Dictionary(931 unique tokens: ['dance pop', 'hip hop', 'hip pop', 'pop', 'pop rap']...)
2018-05-04 05:36:04,201 : INFO : adding document #30000 to Dictionary(1022 unique tokens: ['dance pop', 'hip hop', 'hip pop', 'pop', 'pop rap']...)
2018-05-04 05:36:04,287 : INFO : adding document #40000 to Dictionary(1084 unique tokens: ['dance pop', 'hip hop', 'hip pop', 'pop', 'pop rap']...)
2018-05-04 05:36:04,374 : INFO : adding document #50000 to Dictionary(1138 unique tokens: ['dance pop', 'hip hop', 'hip pop', 'pop', 'pop rap']...)
2018-05-04 05:36:04,469 : INFO : adding document #60000 to Dictionary(1192 unique tokens: ['dance pop', 'hip hop', 'hip pop', 'pop', 'pop rap']...)
2018-05-04 05:36:04,562 : I

2018-05-04 05:36:08,815 : INFO : adding document #560000 to Dictionary(1627 unique tokens: ['dance pop', 'hip hop', 'hip pop', 'pop', 'pop rap']...)
2018-05-04 05:36:08,896 : INFO : adding document #570000 to Dictionary(1631 unique tokens: ['dance pop', 'hip hop', 'hip pop', 'pop', 'pop rap']...)
2018-05-04 05:36:08,977 : INFO : adding document #580000 to Dictionary(1634 unique tokens: ['dance pop', 'hip hop', 'hip pop', 'pop', 'pop rap']...)
2018-05-04 05:36:09,058 : INFO : adding document #590000 to Dictionary(1636 unique tokens: ['dance pop', 'hip hop', 'hip pop', 'pop', 'pop rap']...)
2018-05-04 05:36:09,132 : INFO : adding document #600000 to Dictionary(1637 unique tokens: ['dance pop', 'hip hop', 'hip pop', 'pop', 'pop rap']...)
2018-05-04 05:36:09,209 : INFO : adding document #610000 to Dictionary(1638 unique tokens: ['dance pop', 'hip hop', 'hip pop', 'pop', 'pop rap']...)
2018-05-04 05:36:09,291 : INFO : adding document #620000 to Dictionary(1642 unique tokens: ['dance pop', '

2018-05-04 05:36:13,367 : INFO : adding document #1110000 to Dictionary(1679 unique tokens: ['dance pop', 'hip hop', 'hip pop', 'pop', 'pop rap']...)
2018-05-04 05:36:13,448 : INFO : adding document #1120000 to Dictionary(1679 unique tokens: ['dance pop', 'hip hop', 'hip pop', 'pop', 'pop rap']...)
2018-05-04 05:36:13,519 : INFO : adding document #1130000 to Dictionary(1679 unique tokens: ['dance pop', 'hip hop', 'hip pop', 'pop', 'pop rap']...)
2018-05-04 05:36:13,592 : INFO : adding document #1140000 to Dictionary(1679 unique tokens: ['dance pop', 'hip hop', 'hip pop', 'pop', 'pop rap']...)
2018-05-04 05:36:13,666 : INFO : adding document #1150000 to Dictionary(1679 unique tokens: ['dance pop', 'hip hop', 'hip pop', 'pop', 'pop rap']...)
2018-05-04 05:36:13,740 : INFO : adding document #1160000 to Dictionary(1679 unique tokens: ['dance pop', 'hip hop', 'hip pop', 'pop', 'pop rap']...)
2018-05-04 05:36:13,817 : INFO : adding document #1170000 to Dictionary(1679 unique tokens: ['dance 

2018-05-04 05:36:17,621 : INFO : adding document #1660000 to Dictionary(1684 unique tokens: ['dance pop', 'hip hop', 'hip pop', 'pop', 'pop rap']...)
2018-05-04 05:36:17,698 : INFO : adding document #1670000 to Dictionary(1684 unique tokens: ['dance pop', 'hip hop', 'hip pop', 'pop', 'pop rap']...)
2018-05-04 05:36:17,776 : INFO : adding document #1680000 to Dictionary(1684 unique tokens: ['dance pop', 'hip hop', 'hip pop', 'pop', 'pop rap']...)
2018-05-04 05:36:17,842 : INFO : adding document #1690000 to Dictionary(1685 unique tokens: ['dance pop', 'hip hop', 'hip pop', 'pop', 'pop rap']...)
2018-05-04 05:36:17,918 : INFO : adding document #1700000 to Dictionary(1685 unique tokens: ['dance pop', 'hip hop', 'hip pop', 'pop', 'pop rap']...)
2018-05-04 05:36:17,997 : INFO : adding document #1710000 to Dictionary(1685 unique tokens: ['dance pop', 'hip hop', 'hip pop', 'pop', 'pop rap']...)
2018-05-04 05:36:18,079 : INFO : adding document #1720000 to Dictionary(1685 unique tokens: ['dance 

2018-05-04 05:36:21,761 : INFO : adding document #2210000 to Dictionary(1686 unique tokens: ['dance pop', 'hip hop', 'hip pop', 'pop', 'pop rap']...)
2018-05-04 05:36:21,822 : INFO : adding document #2220000 to Dictionary(1686 unique tokens: ['dance pop', 'hip hop', 'hip pop', 'pop', 'pop rap']...)
2018-05-04 05:36:21,889 : INFO : adding document #2230000 to Dictionary(1686 unique tokens: ['dance pop', 'hip hop', 'hip pop', 'pop', 'pop rap']...)
2018-05-04 05:36:21,961 : INFO : adding document #2240000 to Dictionary(1686 unique tokens: ['dance pop', 'hip hop', 'hip pop', 'pop', 'pop rap']...)
2018-05-04 05:36:22,031 : INFO : adding document #2250000 to Dictionary(1686 unique tokens: ['dance pop', 'hip hop', 'hip pop', 'pop', 'pop rap']...)
2018-05-04 05:36:22,099 : INFO : adding document #2260000 to Dictionary(1686 unique tokens: ['dance pop', 'hip hop', 'hip pop', 'pop', 'pop rap']...)
2018-05-04 05:36:22,117 : INFO : built Dictionary(1686 unique tokens: ['dance pop', 'hip hop', 'hip 

...

2018-05-04 05:36:49,169 : INFO : PROGRESS: saving document #1709000
2018-05-04 05:36:49,176 : INFO : PROGRESS: saving document #1710000
2018-05-04 05:36:49,183 : INFO : PROGRESS: saving document #1711000
2018-05-04 05:36:49,190 : INFO : PROGRESS: saving document #1712000
2018-05-04 05:36:49,196 : INFO : PROGRESS: saving document #1713000
2018-05-04 05:36:49,203 : INFO : PROGRESS: saving document #1714000
2018-05-04 05:36:49,209 : INFO : PROGRESS: saving document #1715000
2018-05-04 05:36:49,217 : INFO : PROGRESS: saving document #1716000
2018-05-04 05:36:49,226 : INFO : PROGRESS: saving document #1717000
2018-05-04 05:36:49,232 : INFO : PROGRESS: saving document #1718000
2018-05-04 05:36:49,240 : INFO : PROGRESS: saving document #1719000
2018-05-04 05:36:49,248 : INFO : PROGRESS: saving document #1720000
2018-05-04 05:36:49,256 : INFO : PROGRESS: saving document #1721000
2018-05-04 05:36:49,264 : INFO : PROGRESS: saving document #1722000
2018-05-04 05:36:49,273 : INFO : PROGRESS: savin

2018-05-04 05:36:50,142 : INFO : PROGRESS: saving document #1830000
2018-05-04 05:36:50,149 : INFO : PROGRESS: saving document #1831000
2018-05-04 05:36:50,157 : INFO : PROGRESS: saving document #1832000
2018-05-04 05:36:50,163 : INFO : PROGRESS: saving document #1833000
2018-05-04 05:36:50,170 : INFO : PROGRESS: saving document #1834000
2018-05-04 05:36:50,177 : INFO : PROGRESS: saving document #1835000
2018-05-04 05:36:50,183 : INFO : PROGRESS: saving document #1836000
2018-05-04 05:36:50,190 : INFO : PROGRESS: saving document #1837000
2018-05-04 05:36:50,197 : INFO : PROGRESS: saving document #1838000
2018-05-04 05:36:50,203 : INFO : PROGRESS: saving document #1839000
2018-05-04 05:36:50,215 : INFO : PROGRESS: saving document #1840000
2018-05-04 05:36:50,223 : INFO : PROGRESS: saving document #1841000
2018-05-04 05:36:50,230 : INFO : PROGRESS: saving document #1842000
2018-05-04 05:36:50,242 : INFO : PROGRESS: saving document #1843000
2018-05-04 05:36:50,253 : INFO : PROGRESS: savin

2018-05-04 05:36:50,998 : INFO : PROGRESS: saving document #1951000
2018-05-04 05:36:51,004 : INFO : PROGRESS: saving document #1952000
2018-05-04 05:36:51,011 : INFO : PROGRESS: saving document #1953000
2018-05-04 05:36:51,020 : INFO : PROGRESS: saving document #1954000
2018-05-04 05:36:51,028 : INFO : PROGRESS: saving document #1955000
2018-05-04 05:36:51,034 : INFO : PROGRESS: saving document #1956000
2018-05-04 05:36:51,041 : INFO : PROGRESS: saving document #1957000
2018-05-04 05:36:51,048 : INFO : PROGRESS: saving document #1958000
2018-05-04 05:36:51,055 : INFO : PROGRESS: saving document #1959000
2018-05-04 05:36:51,064 : INFO : PROGRESS: saving document #1960000
2018-05-04 05:36:51,071 : INFO : PROGRESS: saving document #1961000
2018-05-04 05:36:51,078 : INFO : PROGRESS: saving document #1962000
2018-05-04 05:36:51,085 : INFO : PROGRESS: saving document #1963000
2018-05-04 05:36:51,093 : INFO : PROGRESS: saving document #1964000
2018-05-04 05:36:51,100 : INFO : PROGRESS: savin

2018-05-04 05:36:51,893 : INFO : PROGRESS: saving document #2072000
2018-05-04 05:36:51,903 : INFO : PROGRESS: saving document #2073000
2018-05-04 05:36:51,911 : INFO : PROGRESS: saving document #2074000
2018-05-04 05:36:51,922 : INFO : PROGRESS: saving document #2075000
2018-05-04 05:36:51,931 : INFO : PROGRESS: saving document #2076000
2018-05-04 05:36:51,940 : INFO : PROGRESS: saving document #2077000
2018-05-04 05:36:51,950 : INFO : PROGRESS: saving document #2078000
2018-05-04 05:36:51,958 : INFO : PROGRESS: saving document #2079000
2018-05-04 05:36:51,965 : INFO : PROGRESS: saving document #2080000
2018-05-04 05:36:51,974 : INFO : PROGRESS: saving document #2081000
2018-05-04 05:36:51,982 : INFO : PROGRESS: saving document #2082000
2018-05-04 05:36:51,990 : INFO : PROGRESS: saving document #2083000
2018-05-04 05:36:51,998 : INFO : PROGRESS: saving document #2084000
2018-05-04 05:36:52,005 : INFO : PROGRESS: saving document #2085000
2018-05-04 05:36:52,012 : INFO : PROGRESS: savin

2018-05-04 05:36:52,786 : INFO : PROGRESS: saving document #2193000
2018-05-04 05:36:52,792 : INFO : PROGRESS: saving document #2194000
2018-05-04 05:36:52,797 : INFO : PROGRESS: saving document #2195000
2018-05-04 05:36:52,803 : INFO : PROGRESS: saving document #2196000
2018-05-04 05:36:52,809 : INFO : PROGRESS: saving document #2197000
2018-05-04 05:36:52,821 : INFO : PROGRESS: saving document #2198000
2018-05-04 05:36:52,829 : INFO : PROGRESS: saving document #2199000
2018-05-04 05:36:52,836 : INFO : PROGRESS: saving document #2200000
2018-05-04 05:36:52,842 : INFO : PROGRESS: saving document #2201000
2018-05-04 05:36:52,849 : INFO : PROGRESS: saving document #2202000
2018-05-04 05:36:52,855 : INFO : PROGRESS: saving document #2203000
2018-05-04 05:36:52,861 : INFO : PROGRESS: saving document #2204000
2018-05-04 05:36:52,870 : INFO : PROGRESS: saving document #2205000
2018-05-04 05:36:52,879 : INFO : PROGRESS: saving document #2206000
2018-05-04 05:36:52,886 : INFO : PROGRESS: savin

## Train model

In [51]:
import logging, gensim
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
#id2word = gensim.corpora.Dictionary.load_from_text('genres.dict')
mm = gensim.corpora.MmCorpus('genres.mm')
lda = gensim.models.ldamodel.LdaModel(corpus=mm, id2word=dictionary,
                                      alpha='auto', eta='auto', 
                                      num_topics=20, update_every=1,
                                      chunksize=10000, passes=5)
lda.save('genre_lda')

2018-05-04 05:40:56,610 : INFO : loaded corpus index from genres.mm.index
2018-05-04 05:40:56,610 : INFO : initializing cython corpus reader from genres.mm
2018-05-04 05:40:56,611 : INFO : accepted corpus with 2262292 documents, 1686 features, 6507084 non-zero entries
2018-05-04 05:40:56,611 : INFO : using autotuned alpha, starting with [0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05]
2018-05-04 05:40:56,612 : INFO : using serial LDA version on this node
2018-05-04 05:40:56,617 : INFO : running online (multi-pass) LDA training, 20 topics, 5 passes over the supplied corpus of 2262292 documents, updating model once every 10000 documents, evaluating perplexity every 100000 documents, iterating 50x with a convergence threshold of 0.001000
2018-05-04 05:40:56,652 : INFO : PROGRESS: pass 0, at document #10000/2262292
2018-05-04 05:40:58,690 : INFO : optimized alpha [0.047022812, 0.04667242, 0.047101587, 0.048393283, 0.04

2018-05-04 05:41:03,730 : INFO : optimized alpha [0.043292668, 0.04654568, 0.041666627, 0.042137176, 0.043716714, 0.043702673, 0.04310333, 0.04725405, 0.04853442, 0.043012973, 0.043794397, 0.050480157, 0.04068978, 0.04122292, 0.043784954, 0.04102665, 0.045414764, 0.048506416, 0.041385848, 0.04156512]
2018-05-04 05:41:03,730 : INFO : merging changes from 10000 documents into a model of 2262292 documents
2018-05-04 05:41:03,736 : INFO : topic #12 (0.041): 0.133*"pop punk" + 0.090*"emo" + 0.077*"modern rock" + 0.075*"punk" + 0.068*"dreamo" + 0.053*"screamo" + 0.044*"skate punk" + 0.039*"post-hardcore" + 0.034*"alternative emo" + 0.024*"melodic hardcore"
2018-05-04 05:41:03,736 : INFO : topic #15 (0.041): 0.109*"underground hip hop" + 0.077*"vapor trap" + 0.072*"trap music" + 0.053*"canadian ccm" + 0.048*"rap" + 0.045*"christian rock" + 0.038*"christian alternative rock" + 0.035*"operatic pop" + 0.032*"southern hip hop" + 0.029*"deep trap"
2018-05-04 05:41:03,737 : INFO : topic #17 (0.049)

2018-05-04 05:41:08,075 : INFO : topic #8 (0.049): 0.142*"rap" + 0.135*"hip hop" + 0.125*"pop rap" + 0.092*"southern hip hop" + 0.069*"trap music" + 0.069*"gangster rap" + 0.051*"dirty south rap" + 0.043*"pop" + 0.035*"hardcore hip hop" + 0.025*"alternative hip hop"
2018-05-04 05:41:08,077 : INFO : topic #17 (0.049): 0.175*"pop" + 0.168*"dance pop" + 0.152*"post-teen pop" + 0.060*"classical" + 0.037*"hollywood" + 0.037*"romantic era" + 0.035*"viral pop" + 0.028*"show tunes" + 0.026*"broadway" + 0.017*"soundtrack"
2018-05-04 05:41:08,078 : INFO : topic #11 (0.051): 0.112*"contemporary country" + 0.102*"country road" + 0.091*"country" + 0.072*"alternative metal" + 0.065*"modern country rock" + 0.062*"nu metal" + 0.052*"post-grunge" + 0.050*"rap metal" + 0.042*"regional mexican" + 0.035*"redneck"
2018-05-04 05:41:08,079 : INFO : topic diff=1.630201, rho=0.333333
2018-05-04 05:41:10,923 : INFO : -5.226 per-word bound, 37.4 perplexity estimate based on a held-out corpus of 10000 documents w

2018-05-04 05:41:16,667 : INFO : topic #11 (0.050): 0.085*"alternative metal" + 0.083*"contemporary country" + 0.073*"country road" + 0.071*"nu metal" + 0.065*"country" + 0.063*"post-grunge" + 0.053*"rap metal" + 0.048*"regional mexican" + 0.045*"modern country rock" + 0.041*"rap rock"
2018-05-04 05:41:16,668 : INFO : topic diff=1.235910, rho=0.277350
2018-05-04 05:41:16,700 : INFO : PROGRESS: pass 0, at document #140000/2262292
2018-05-04 05:41:18,069 : INFO : optimized alpha [0.04183149, 0.046726275, 0.038450204, 0.037829652, 0.038061175, 0.044304095, 0.04082273, 0.04968503, 0.04827559, 0.03925426, 0.040161517, 0.050213754, 0.03808673, 0.036953487, 0.042024873, 0.035463143, 0.04266637, 0.048704695, 0.03837549, 0.03849044]
2018-05-04 05:41:18,071 : INFO : merging changes from 10000 documents into a model of 2262292 documents
2018-05-04 05:41:18,081 : INFO : topic #15 (0.035): 0.184*"underground hip hop" + 0.103*"vapor trap" + 0.070*"trap music" + 0.045*"deep trap" + 0.041*"drill" + 0.

2018-05-04 05:41:22,840 : INFO : optimized alpha [0.04169843, 0.04641274, 0.03795888, 0.03722448, 0.036508095, 0.0442484, 0.040288296, 0.04987104, 0.04753926, 0.038712293, 0.039489575, 0.049375758, 0.038139142, 0.03611261, 0.04185893, 0.034218017, 0.04183893, 0.048347753, 0.037377477, 0.037387077]
2018-05-04 05:41:22,841 : INFO : merging changes from 10000 documents into a model of 2262292 documents
2018-05-04 05:41:22,848 : INFO : topic #15 (0.034): 0.209*"underground hip hop" + 0.118*"vapor trap" + 0.084*"trap music" + 0.052*"operatic pop" + 0.050*"deep trap" + 0.046*"drill" + 0.031*"canadian ccm" + 0.025*"electropowerpop" + 0.022*"christian punk" + 0.021*"idol"
2018-05-04 05:41:22,848 : INFO : topic #13 (0.036): 0.093*"k-pop" + 0.092*"indie pop rap" + 0.090*"deep underground hip hop" + 0.077*"indie anthem-folk" + 0.076*"chamber pop" + 0.065*"indie folk" + 0.049*"korean pop" + 0.048*"stomp and holler" + 0.045*"aussietronica" + 0.041*"deep australian indie"
2018-05-04 05:41:22,849 : I

...

2018-05-04 05:59:04,848 : INFO : merging changes from 10000 documents into a model of 2262292 documents
2018-05-04 05:59:04,857 : INFO : topic #15 (0.020): 0.051*"chicano rap" + 0.042*"operatic pop" + 0.042*"italian pop" + 0.035*"laiko" + 0.034*"entehno" + 0.033*"turkish rock" + 0.033*"underground hip hop" + 0.031*"turkish pop" + 0.026*"tango" + 0.026*"christian punk"
2018-05-04 05:59:04,858 : INFO : topic #13 (0.024): 0.070*"stomp and holler" + 0.052*"smooth jazz" + 0.041*"deep underground hip hop" + 0.041*"new americana" + 0.039*"folk-pop" + 0.039*"k-pop" + 0.038*"indie folk" + 0.038*"danish pop" + 0.036*"chamber pop" + 0.035*"korean pop"
2018-05-04 05:59:04,859 : INFO : topic #14 (0.041): 0.055*"latin" + 0.049*"early modern classical" + 0.044*"tropical" + 0.039*"latin pop" + 0.036*"salsa" + 0.032*"latin jazz" + 0.031*"mpb" + 0.031*"mambo" + 0.031*"latin hip hop" + 0.029*"minimal techno"
2018-05-04 05:59:04,860 : INFO : topic #19 (0.043): 0.052*"trance" + 0.051*"progressive trance" +

2018-05-04 05:59:08,971 : INFO : topic #14 (0.041): 0.055*"latin" + 0.051*"early modern classical" + 0.047*"tropical" + 0.040*"latin pop" + 0.039*"salsa" + 0.037*"latin jazz" + 0.035*"mambo" + 0.030*"mpb" + 0.029*"latin hip hop" + 0.028*"merengue"
2018-05-04 05:59:08,971 : INFO : topic #19 (0.043): 0.055*"trance" + 0.053*"progressive trance" + 0.051*"progressive house" + 0.050*"uplifting trance" + 0.034*"speed metal" + 0.026*"piano blues" + 0.026*"healing" + 0.026*"classify" + 0.022*"meditation" + 0.021*"latin christian"
2018-05-04 05:59:08,972 : INFO : topic #0 (0.047): 0.056*"classic rock" + 0.050*"rock" + 0.046*"mellow gold" + 0.045*"folk rock" + 0.041*"album rock" + 0.039*"singer-songwriter" + 0.030*"roots rock" + 0.029*"soft rock" + 0.029*"blues-rock" + 0.028*"folk"
2018-05-04 05:59:08,973 : INFO : topic diff=inf, rho=0.065763
2018-05-04 05:59:08,998 : INFO : PROGRESS: pass 4, at document #2120000/2262292
2018-05-04 05:59:09,724 : INFO : optimized alpha [0.047395915, 0.027784614, 

2018-05-04 05:59:12,093 : INFO : topic diff=inf, rho=0.065763
2018-05-04 05:59:12,129 : INFO : PROGRESS: pass 4, at document #2160000/2262292
2018-05-04 05:59:12,825 : INFO : optimized alpha [0.047503766, 0.027757233, 0.035910163, 0.027974766, 0.02962352, 0.033366572, 0.032327283, 0.037900366, 0.027667357, 0.02991716, 0.03593381, 0.031343896, 0.027638715, 0.024244413, 0.041213103, 0.019815061, 0.035208363, 0.038254425, 0.024705091, 0.042915057]
2018-05-04 05:59:12,831 : INFO : merging changes from 10000 documents into a model of 2262292 documents
2018-05-04 05:59:12,840 : INFO : topic #15 (0.020): 0.044*"chicano rap" + 0.043*"italian pop" + 0.040*"operatic pop" + 0.040*"laiko" + 0.040*"entehno" + 0.035*"underground hip hop" + 0.035*"turkish pop" + 0.029*"turkish rock" + 0.028*"tango" + 0.026*"christian punk"
2018-05-04 05:59:12,841 : INFO : topic #13 (0.024): 0.062*"stomp and holler" + 0.051*"smooth jazz" + 0.047*"deep underground hip hop" + 0.039*"new americana" + 0.037*"k-pop" + 0.03

2018-05-04 05:59:16,916 : INFO : topic #15 (0.020): 0.045*"laiko" + 0.045*"operatic pop" + 0.044*"entehno" + 0.043*"italian pop" + 0.037*"chicano rap" + 0.034*"underground hip hop" + 0.034*"turkish pop" + 0.030*"tango" + 0.029*"classic persian pop" + 0.027*"turkish rock"
2018-05-04 05:59:16,917 : INFO : topic #13 (0.024): 0.064*"stomp and holler" + 0.055*"smooth jazz" + 0.044*"deep underground hip hop" + 0.041*"new americana" + 0.039*"chamber pop" + 0.036*"folk-pop" + 0.036*"k-pop" + 0.033*"indie folk" + 0.032*"australian pop" + 0.031*"korean pop"
2018-05-04 05:59:16,917 : INFO : topic #14 (0.041): 0.055*"latin" + 0.049*"tropical" + 0.047*"early modern classical" + 0.041*"latin pop" + 0.035*"salsa" + 0.034*"latin jazz" + 0.032*"mpb" + 0.032*"mambo" + 0.028*"samba" + 0.028*"merengue"
2018-05-04 05:59:16,918 : INFO : topic #19 (0.043): 0.053*"trance" + 0.053*"progressive trance" + 0.050*"uplifting trance" + 0.049*"progressive house" + 0.031*"speed metal" + 0.029*"classify" + 0.026*"piano

2018-05-04 05:59:19,919 : INFO : topic #19 (0.043): 0.051*"trance" + 0.051*"progressive trance" + 0.047*"uplifting trance" + 0.047*"progressive house" + 0.031*"speed metal" + 0.027*"classify" + 0.026*"healing" + 0.024*"piano blues" + 0.022*"latin christian" + 0.022*"meditation"
2018-05-04 05:59:19,920 : INFO : topic #0 (0.047): 0.056*"classic rock" + 0.055*"rock" + 0.051*"mellow gold" + 0.046*"album rock" + 0.045*"folk rock" + 0.038*"singer-songwriter" + 0.032*"soft rock" + 0.032*"roots rock" + 0.028*"blues-rock" + 0.028*"folk"
2018-05-04 05:59:19,922 : INFO : topic diff=inf, rho=0.065763
2018-05-04 05:59:19,947 : INFO : PROGRESS: pass 4, at document #2250000/2262292
2018-05-04 05:59:20,656 : INFO : optimized alpha [0.047462814, 0.027653636, 0.035905544, 0.028021755, 0.029655654, 0.033255644, 0.032196116, 0.037725102, 0.027529422, 0.029756753, 0.03600942, 0.031220559, 0.02751272, 0.024124755, 0.041469634, 0.01987704, 0.03530936, 0.038561184, 0.024774766, 0.04304364]
2018-05-04 05:59:20

## test model

In [55]:
#lda = gensim.models.ldamodel.LdaModel.load('genre_lda')
doc_bow = dictionary.doc2bow(turi2genres['0UaMYEvWZi0ZqiDOoHU3YI'])
topic_mix= lda[doc_bow]

[(8, 0.33578992), (9, 0.5955591)]
9


In [58]:
turi2genres = cpick.load('../data/interim/track_uri2genres.pkl.bz2')
turi2gtopic={}
for turi,genres in turi2genres.items():
    topic_mix= lda[dictionary.doc2bow(genres)]
    w = [i[1] for i in topic_mix]
    w_mean = np.mean(w)
    w_max = np.max(w)
    if  w_mean+0.01 > w_max:
        topic=21
    else:
        topic =  [i[0] for i in topic_mix if i[1]==w_max][0]
        
    turi2gtopic[turi]=topic
print(glance_dict(turi2gtopic))
cpick.save('../data/interim/track_uri2gtopics.pkl.bz2',turi2gtopic)

{'0UaMYEvWZi0ZqiDOoHU3YI': 9, '6I9VzXrHxO9rA9A5euc8Ak': 9, '0WqIKmW4BTrj3eJFmnCKMv': 9}
Saving to ../data/interim/track_uri2gtopics.pkl.bz2
